In [1]:
import streamlit as st
from streamlit_chat import message
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.vectorstores import FAISS
import tempfile
import json

In [2]:
f = open("apikey.json")
data = json.load(f)
key = data["openaikey"]

In [3]:
user_api_key = st.sidebar.text_input(
    label="#### Your OpenAI API key 👇",
    placeholder=key,
    type="password")

2023-11-03 15:20:50.131 
  command:

    streamlit run C:\Users\joshg\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\ipykernel_launcher.py [ARGUMENTS]


In [4]:
loader = CSVLoader(file_path="./RAW_recipes.csv", encoding="utf-8", csv_args={
            'delimiter': ','})
data = loader.load()
st.write(data)

In [4]:
embeddings = OpenAIEmbeddings(openai_api_key=key)

In [5]:
vectorstore = FAISS.from_documents(data, embeddings)

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: You exceeded your current quota, please check your plan and billing details..
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-embedding-ada-002 in organization org-CKW62Pbjl6M1w5R0NleHSedc on tokens per min. Limit: 1000000 / min. Current: 626680 / min. Visit https://platform.openai.com/account/rate-limits to learn more..
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-embedding-ada-002 in organization org-CKW62Pbjl6M1w5R0NleHSedc on tokens per min. Limit: 1000000 / min. Current: 864540 / min. Visit https://platform.openai.com/account/rate-limits to learn more..
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds a

In [6]:
vectorstore.save_local("faiss_dump")

In [10]:
db = FAISS.load_local("faiss_dump", embeddings)
documents = db.similarity_search(query = "recipe with eggs, salt, butter, and sugar", k = 1)
[doc.page_content for doc in documents]

["name: egg butter\nid: 181010\nminutes: 10\ncontributor_id: 269843\nsubmitted: 2006-08-08\ntags: ['15-minutes-or-less', 'time-to-make', 'course', 'preparation', 'occasion', 'low-protein', 'sauces', 'breakfast', 'condiments-etc', 'dietary', 'low-sodium', 'comfort-food', 'sweet-sauces', 'low-in-something', 'brunch', 'taste-mood']\nnutrition: [568.1, 37.0, 319.0, 7.0, 3.0, 74.0, 29.0]\nn_steps: 6\nsteps: ['melt butter in a medium sized sauce pan and cool', 'stir in the egg , sugar , corn syrup & water', 'bring the mixture to a boil and cook for', '1 minute', 'remove from heat and stir in the vanilla', 'cover and set aside to cool while preparing pancakes or waffles or biscuits']\ndescription: this is a tasty syrup for biscuits, pancakes or waffles.  it also goes well on angel food cake.  you can keep it in the refrigerator for a few days if needed.\ningredients: ['butter', 'egg', 'sugar', 'light corn syrup', 'water', 'vanilla']\nn_ingredients: 6"]

In [7]:
db = FAISS.load_local("faiss_dump",embeddings)
chain = ConversationalRetrievalChain.from_llm(
llm = ChatOpenAI(temperature=0.15,model_name='gpt-3.5-turbo', openai_api_key=key),
retriever=db.as_retriever())

In [8]:
def conversational_chat(query):
    
    result = chain({"question": query, 
    "chat_history": st.session_state['history']})
    st.session_state['history'].append((query, result["answer"]))
    
    return result["answer"]

2023-11-03 15:25:32.315 Session state does not function when running a script without `streamlit run`


KeyError: 'st.session_state has no key "history". Did you forget to initialize it? More info: https://docs.streamlit.io/library/advanced-features/session-state#initialization'